球諧函數（Spherical Harmonics）:
$$
Y_\ell^m(\theta, \phi) =
\sqrt{\frac{(2\ell+1)}{4\pi}\frac{(\ell-m)!}{(\ell+m)!}}
\, P_\ell^m(\cos\theta)\, e^{i m \phi},
$$
其中 $\theta\in[0,\pi]$（极角/天顶角）, $\ell>=0$, $m \in[−\ell,\ell]$，$\phi\in[0,2\pi)$（方位角），$P_\ell^m$ 为连带勒让德多项式:
$$
P_\ell^m(x) = (1-x^2)^{\tfrac{m}{2}} 
\frac{d^m}{dx^m} P_\ell(x),
$$

把「复数球谐基 $Y_\ell^m$」转换为「实数基 $Y_{\ell m}^{\text{real}}$」的公式
$$
Y_{\ell m}^{\text{real}}(\theta,\phi) =
\begin{cases}
\sqrt{2}\,(-1)^m\,\mathrm{Im}\,Y_\ell^{|m|}(\theta,\phi), & m < 0, \\[6pt]
Y_\ell^{0}(\theta,\phi), & m = 0, \\[6pt]
\sqrt{2}\,(-1)^m\,\mathrm{Re}\,Y_\ell^{m}(\theta,\phi), & m > 0.
\end{cases}
$$


到 $\ell =3$的 16 个实数 SH 基, 同样采用 Sloan/Green 约定（全归一化，直接代入单位方向向量 $(x, y, z)$ 满足 $x^2 + y^2 + z^2 =1$:
$$
\begin{aligned}
Y_{0}^{0} &= 0.2820947918 \\[6pt]
\end{aligned}
$$
$$
\begin{aligned}
Y_{1}^{-1} &= 0.4886025119 \, y \\ 
Y_{1}^{0}  &= 0.4886025119 \, z, \\ 
Y_{1}^{1}  &= 0.4886025119 \, x \\
\end{aligned}
$$
$$
\begin{aligned}
Y_{2}^{-2} &= 1.0925484306 \, x y \\
Y_{2}^{-1} &= 1.0925484306 \, y z \\
Y_{2}^{0}  &= 0.3153915653 \, (3z^2 - 1) \\
Y_{2}^{1}  &= 1.0925484306 \, x z \\
Y_{2}^{2}  &= 0.5462742153 \, (x^2 - y^2) \\
\end{aligned}
$$
$$
\begin{aligned}
Y_{3}^{-3} &= 0.5900435899 \, y \, (3x^2 - y^2), \\
Y_{3}^{-2} &= 2.8906114426 \, x y z, \\
Y_{3}^{-1} &= 0.4570457995 \, y \, (5z^2 - 1), \\
Y_{3}^{0}  &= 0.3731763326 \, z \, (5z^2 - 3), \\
Y_{3}^{1}  &= 0.4570457995 \, x \, (5z^2 - 1), \\
Y_{3}^{2}  &= 1.4453057213 \, z \, (x^2 - y^2), \\
Y_{3}^{3}  &= 0.5900435899 \, x \, (x^2 - 3y^2).
\end{aligned}
$$

我们在渲染中通常使用球谐函数展开光照：
$$
L(n) \approx \sum_{\ell=0}^{L} \sum_{m=-\ell}^{\ell} c_{\ell m}\, Y_{\ell m}(n)
$$
这里的 $c_{\ell m}$ 就是系数，表示函数在球谐基上的投影。

---

### 1. 数学定义

系数 $c_{\ell m}$ 是将一个函数（例如环境光照 $L(\omega)$，$\omega$ 是单位方向向量）投影到球谐基 $Y_{\ell m}$ 上得到的：

$$
c_{\ell m} = \int_{\Omega} L(\omega)\, Y_{\ell m}(\omega) \, d\omega
$$
- $\Omega$：单位球面  
- $L(\omega)$：某个方向的入射辐射度 (radiance)，可以来自 HDR 环境贴图  
- $Y_{\ell m}(\omega)$：球谐基函数  

这个积分就是在「求函数在基函数方向上的投影」。

### 2. 在渲染里的物理意义

- $c_{\ell m}$ 存储了 **环境光照场** 的信息。  
- 如果是 RGB 光照，每个 $c_{\ell m}$ 实际上是一个三维向量 $(R, G, B)$。  
- 把整个 HDR 环境图压缩成几十个 $c_{\ell m}$，就能用很低的成本在任意方向上重建光照。

---

### 3. 实际用法

**预计算阶段**：

1. 从环境贴图 $L(\omega)$ 计算 $c_{\ell m}$（做积分，常用采样近似）。  
2. 存储下来。例如：
   - 二阶 SH（$\ell \le 3$） → 16 个系数 即 $\mathbf{c}$， 每个都是三维向量 $(R, G, B)$。

**渲染阶段**：

1. 给定法线方向 $n$，调用 `eval_sh_basis(n)` 得到基向量 $\mathbf{y}(n)$。  
    - 例如，$n = (x, y, z)$，它就是把方向 $n$ 映射到一个 **16 维的向量**：$\mathbf{y}(n) =\begin{bmatrix}Y_{0}^{0}(n), &Y_{1}^{-1}(n), &Y_{1}^{0}(n), &\dots, &Y_{3}^{3}(n)\end{bmatrix}
$
2. 直接点积：
$$
L(n) \approx \mathbf{c} \cdot \mathbf{y}(n)
$$
3. 得到当前方向上的光照。

In [ ]:
def eval_sh_basis(n):
    x,y,z = n
    return np.array([
        0.282095,                          # l=0,m=0
        0.488603*y,                        # l=1,m=-1
        0.488603*z,                        # l=1,m=0
        0.488603*x,                        # l=1,m=1
        1.092548*x*y,                      # l=2,m=-2
        1.092548*y*z,                      # l=2,m=-1
        0.315392*(3*z*z - 1),              # l=2,m=0
        1.092548*x*z,                      # l=2,m=1
        0.546274*(x*x - y*y),              # l=2,m=2
        0.590044*y*(3*x*x - y*y),          # l=3,m=-3
        2.890611*x*y*z,                    # l=3,m=-2
        0.457046*y*(5*z*z - 1),            # l=3,m=-1
        0.373176*z*(5*z*z - 3),            # l=3,m=0
        0.457046*x*(5*z*z - 1),            # l=3,m=1
        1.445306*z*(x*x - y*y),            # l=3,m=2
        0.590044*x*(x*x - 3*y*y),          # l=3,m=3
    ])